<a href="https://colab.research.google.com/github/sunshineluyao/ESG_with_benchmark/blob/main/Categorization/ESG_Categorization_Jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### import packages
import pandas as pd
import numpy as np
### show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
### Import data
df_Jan= pd.read_csv("https://raw.githubusercontent.com/sunshineluyao/ESG_with_benchmark/main/rawdata/SP1500_Jan.csv")
df_Jan.head()

In [ ]:
df_Jan=df_Jan.dropna()

In [ ]:
df_Jan=df_Jan.set_index('Symbol',drop=True)
df_Jan.head()

In [ ]:
df_Jan.index.name=None
df_Jan.head()

In [ ]:
### formating data so that it can be calculated
df_Jan[['Size','Common Shares Outstdg','MktVal Co']]=df_Jan[['Size','Common Shares Outstdg','MktVal Co']].apply(lambda x: x.str.replace(',', ''), axis=1)
df_Jan.head()

In [ ]:
df_Jan[['Size', 'Book Value Per Shr Tangble','Common Shares Outstdg','MktVal Co','Momentum']] = df_Jan[['Size', 'Book Value Per Shr Tangble','Common Shares Outstdg','MktVal Co','Momentum']].astype('float')
df_Jan.head()

In [ ]:
### Generate the new variable:BM_Ratio
df_Jan['BM_Ratio'] = df_Jan['Book Value Per Shr Tangble']*df_Jan['Common Shares Outstdg']/df_Jan['MktVal Co']
df_Jan.head()

In [ ]:
### Generate the categories by quintile cut and ESG rating
df_Jan['Size_Index']=pd.qcut(df_Jan['Size'],5,labels=np.arange(1,6,1))
df_Jan['Momentum_Index']=pd.qcut(df_Jan['Momentum'],5,labels=np.arange(1,6,1))
df_Jan['BM_Ratio_Index']=pd.qcut(df_Jan['BM_Ratio'],5,labels=np.arange(1,6,1))
df_Jan['ESG_Index']=np.where(df_Jan['MSCI ESG Rtg Overall Company Rating'].isin(['A','AA','AAA']), 'High','Low')
df_Jan.head()

In [ ]:
df_Jan_Overview=df_Jan[['Size_Index','Momentum_Index','BM_Ratio_Index','ESG_Index','Monthly Return']]

In [ ]:
df_Jan_Overview_Benchmark = df_Jan_Overview.groupby(by=['Size_Index','Momentum_Index','BM_Ratio_Index',],group_keys=True,sort=True,as_index=False).mean('Monthly Return')

In [ ]:
df_Jan_Overview_Benchmark.head()

In [ ]:
df_Jan_Overview_ESG_High = df_Jan_Overview[df_Jan_Overview['ESG_Index']=='High']
df_Jan_Overview_ESG_High.head()

In [ ]:
df_Jan_Overveiw_ESG_High_Mean = df_Jan_Overview_ESG_High.groupby(by=['Size_Index','Momentum_Index','BM_Ratio_Index',],group_keys=True,sort=True,as_index=False).mean('Monthly Return')

In [ ]:
df_Jan_Overveiw_ESG_High_Mean.head()

In [ ]:
df_Jan_Overview_Benchmark['ESG_High_Return']= df_Jan_Overveiw_ESG_High_Mean['Monthly Return']
df_Jan_Overview_Benchmark.head()

In [ ]:
df_Jan_Overview_Benchmark['Abnormal Return']=df_Jan_Overview_Benchmark['ESG_High_Return']-df_Jan_Overview_Benchmark['Monthly Return']
df_Jan_Overview_Benchmark.head()

In [ ]:
df_Jan_Overview_Benchmark['Month']='Jan'
df_Jan_Overview_Benchmark.head()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.box(df_Jan_Overview_Benchmark, y="Abnormal Return",
                   hover_data=df_Jan_Overview_Benchmark.columns)
fig.show()

In [ ]:
df_Jan_Overview_Benchmark.to_csv('Overview_Jan.csv')